In [11]:
from code_context_manager import *



In [ ]:
from code_context_manager import CodeContextManager
from code_context_manager import CodeFile
# Initialize with persistence file
manager = CodeContextManager("code_context.json")



In [ ]:

with open("test_form.py", "r", encoding="utf-8") as f:
    content = f.read()

manager.update_file(
    name="test_form.py",
    content=content,
    dependencies={},
    description="A simple module"
)


In [ ]:
import os
# walk the directory and add all the python files to the context
for root, dirs, files in os.walk("."):
    for file in files:
        if file.endswith(".py"):
            with open(os.path.join(root, file), "r", encoding="utf-8") as f:
                content = f.read()
                manager.update_file(name=file, content=content, dependencies={}, description="A simple module")



In [ ]:
# test if the code is valid
print(manager.is_code_valid(content))




In [17]:
from rich import print as rr


In [ ]:
for file in manager.files :
    # get a the python code from the text from the ``python code`` block
    rr(file)
    # find the first ``python code`` block
    # rr(code_text)

    # rr(manager.is_code_valid(code_text))
    rr(code_text)
    # rr(f"{file[name]}: {is_valid}")
    rr("-"*110)


In [ ]:

for file in manager.get_context_messages():
    # get a the python code from the text from the ``python code`` block
    code_text = file["content"][0]["text"]
    # find the first ``python code`` block
    # rr(code_text)
    code_start = code_text.find("```python")
    rr(code_start)
    code_end = code_text[code_start+9:].find("```")
    rr(code_end)
    code_text = code_text[code_start+9:code_start+code_end+8]
    # rr(code_context_manager.is_code_valid(code_text))
    # rr(code_text)
    rr(manager.extract_dependencies(code_text))
    # rr(f"{file[name]}: {is_valid}")
    rr("-"*110)
# print(code_context_manager.is_code_valid(manager.files[file]["content"]))



In [23]:
import pandas as pd
from pathlib import Path

DEF_PREFIXES = ['def ' , 'async def ']
NEWLINE = '\n'

def get_function_name(code):
    """
    Extract function name from a line beginning with def or async def.
    """
    for prefix in DEF_PREFIXES:
        if code.startswith(prefix):
            return code[len(prefix): code.index('(')]


def get_until_no_space(all_lines, i):
    """
    Get all lines until a line outside the function definition is found.
    """
    ret = [all_lines[i]]
    for j in range(i + 1, len(all_lines)):
        if len(all_lines[j]) == 0 or all_lines[j][0] in [' ', '\t']:
            ret.append(all_lines[j])
        else:
            break
    return NEWLINE.join(ret)


def get_functions(filepath):
    """
    Get all functions in a Python file.
    """
    with open(filepath, 'r') as file:
        all_lines = file.read().replace('\r', NEWLINE).split(NEWLINE)
        for i, l in enumerate(all_lines):
            for prefix in DEF_PREFIXES:
                if l.startswith(prefix):
                    code = get_until_no_space(all_lines, i)
                    function_name = get_function_name(code)
                    yield {
                        code: code,
                        function_name: function_name,
                        filepath: filepath,
                    }
                    break


def extract_functions_from_repo(code_root):
    """
    Extract all .py functions from the repository.
    """
    code_files = list(code_root.glob("**/*.py"))

    num_files = len(code_files)
    print(f"Total number of .py files: {num_files}")

    if num_files == 0:
        print("Verify openai-python repo exists and code_root is set correctly.")
        return None

    all_funcs = [
        func
        for code_file in code_files
        for func in get_functions(str(code_file))
    ]

    num_funcs = len(all_funcs)
    print(f"Total number of functions extracted: {num_funcs}")

    return all_funcs


In [ ]:
# Set user root directory to the openai-python repository
root_dir = Path.cwd()

# Assumes the openai-python repository exists in the users root directory
code_root = root_dir 
rr(code_root)
# Extract all functions from the repository
all_funcs = extract_functions_from_repo(code_root)


In [ ]:
manager.update_file(
    name="test_form.py",
    content=content,
    dependencies={},
    description="A simple module"
)


In [ ]:
(all_funcs)


In [ ]:
for func in all_funcs:
    # manager.update_file(
    #     name=func["filepath"],
    #     content=func["code"],
    #     dependencies={},
    #     description=func["function_name"]
    # )
    rr(func["filepath"])
    # rr(type(func["function_name"]))
    # rr(type(func["code"]))
    # rr("-"*110)


In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
from pathlib import Path
import pandas as pd
import ast
import inspect

class FunctionInfo(BaseModel):
    code: str
    function_name: str
    filepath: str
    docstring: Optional[str] = None
    args: List[str] = Field(default_factory=list)
    return_annotation: Optional[str] = None

class FileInfo(BaseModel):
    path: Path
    functions: List[FunctionInfo] = Field(default_factory=list)

class CodeAnalyzer(BaseModel):
    code_root: Path
    def_prefixes: List[str] = Field(default=['def ' , 'async def '])
    newline: str = Field(default='\n')
    files: List[FileInfo] = Field(default_factory=list)
    
    class Config:
        arbitrary_types_allowed = True

    def extract_function_details(self, code: str) -> dict:
        """
        Extract function details including docstring using AST
        """
        try:
            tree = ast.parse(code)
            for node in ast.walk(tree):
                if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef)):
                    # Get docstring
                    docstring = ast.get_docstring(node)
                    
                    # Get arguments
                    args = []
                    for arg in node.args.args:
                        args.append(arg.arg)
                    
                    # Get return annotation if it exists
                    return_annotation = None
                    if node.returns:
                        return_annotation = ast.unparse(node.returns)
                    
                    return {
                        "docstring": docstring,
                        "args": args,
                        "return_annotation": return_annotation
                    }
        except SyntaxError:
            # Handle cases where the code snippet might be incomplete
            return {"docstring": None, "args": [], "return_annotation": None}
        
        return {"docstring": None, "args": [], "return_annotation": None}

    def get_function_name(self, code: str) -> Optional[str]:
        """
        Extract function name from a line beginning with def or async def.
        """
        for prefix in self.def_prefixes:
            if code.startswith(prefix):
                return code[len(prefix): code.index('(')]
        return None

    def get_until_no_space(self, all_lines: List[str], i: int) -> str:
        """
        Get all lines until a line outside the function definition is found.
        """
        ret = [all_lines[i]]
        indent_level = len(all_lines[i]) - len(all_lines[i].lstrip())
        
        for j in range(i + 1, len(all_lines)):
            line = all_lines[j]
            if not line.strip():  # Empty line
                ret.append(line)
                continue
                
            current_indent = len(line) - len(line.lstrip())
            if current_indent <= indent_level and line.strip():
                break
                
            ret.append(line)
            
        return self.newline.join(ret)

    def get_functions(self, filepath: str) -> List[FunctionInfo]:
        """
        Get all functions in a Python file.
        """
        with open(filepath, 'r') as file:
            all_lines = file.read().replace('\r', self.newline).split(self.newline)
            functions = []
            i = 0
            while i < len(all_lines):
                line = all_lines[i]
                for prefix in self.def_prefixes:
                    if line.strip().startswith(prefix):
                        code = self.get_until_no_space(all_lines, i)
                        function_name = self.get_function_name(code)
                        if function_name:
                            # Extract additional details using AST
                            details = self.extract_function_details(code)
                            
                            functions.append(
                                FunctionInfo(
                                    code=code,
                                    function_name=function_name,
                                    filepath=filepath,
                                    docstring=details["docstring"],
                                    args=details["args"],
                                    return_annotation=details["return_annotation"]
                                )
                            )
                        break
                i += 1
        return functions

    def analyze_repo(self) -> List[FileInfo]:
        """
        Extract all .py functions from the repository and return them as FileInfo objects.
        """
        code_files = list(self.code_root.glob("**/*.py"))

        num_files = len(code_files)
        print(f"Total number of .py files: {num_files}")

        if num_files == 0:
            print("Verify the repository exists and code_root is set correctly.")
            return []

        self.files = []
        for code_file in code_files:
            file_info = FileInfo(
                path=code_file,
                functions=self.get_functions(str(code_file))
            )
            self.files.append(file_info)

        num_funcs = sum(len(file.functions) for file in self.files)
        print(f"Total number of functions extracted: {num_funcs}")

        return self.files

    def get_function_dict(self) -> Dict[str, List[FunctionInfo]]:
        """
        Returns a dictionary of filename to list of functions
        """
        return {str(file.path): file.functions for file in self.files}

    def to_dataframe(self) -> pd.DataFrame:
        """
        Convert the analyzed data to a pandas DataFrame
        """
        rows = []
        for file in self.files:
            for func in file.functions:
                rows.append({
                    filepath: func.filepath,
                    function_name: func.function_name,
                    code: func.code,
                    docstring: func.docstring,
                    args: func.args,
                    return_annotation: func.return_annotation
                })
        return pd.DataFrame(rows)

    def get_functions_with_docstrings(self) -> List[FunctionInfo]:
        """
        Returns only functions that have docstrings
        """
        return [
            func 
            for file in self.files 
            for func in file.functions 
            if func.docstring
        ]

    def get_functions_missing_docstrings(self) -> List[FunctionInfo]:
        """
        Returns functions that are missing docstrings
        """
        return [
            func 
            for file in self.files 
            for func in file.functions 
            if not func.docstring
        ]

    def get_docstring_coverage_stats(self) -> dict:
        """
        Returns statistics about docstring coverage
        """
        total_functions = sum(len(file.functions) for file in self.files)
        with_docstrings = len(self.get_functions_with_docstrings())
        without_docstrings = len(self.get_functions_missing_docstrings())
        
        return {
            "total_functions": total_functions,
            "with_docstrings": with_docstrings,
            "without_docstrings": without_docstrings,
            "coverage_percentage": (with_docstrings / total_functions * 100) if total_functions > 0 else 0
        }


In [ ]:
# Initialize the analyzer
# Set user root directory to the openai-python repository
root_dir = Path.cwd()

# Assumes the openai-python repository exists in the users root directory
code_root = root_dir 
analyzer = CodeAnalyzer(code_root=code_root,    def_prefixes=[def , async def , cdef , class ],  # Custom prefixes
)

# Analyze the repository
analyzer.analyze_repo()

# Get results in different formats
df = analyzer.to_dataframe()  # As DataFrame
function_dict = analyzer.get_function_dict()  # As Dictionary


# Access the files directly
for file_info in analyzer.files:
    print(f"File: {file_info.path}")
    for function in file_info.functions:
        print(f"  Function: {function.function_name}")


In [ ]:
rr(analyzer.get_functions_missing_docstrings())


In [ ]:
# Initialize and analyze
# analyzer = CodeAnalyzer(code_root=code_root)
analyzer.analyze_repo()

# Get all functions with docstrings
documented_functions = analyzer.get_functions_with_docstrings()

# Get coverage statistics
coverage_stats = analyzer.get_docstring_coverage_stats()
print(f"Docstring coverage: {coverage_stats[coverage_percentage]:.2f}%")

# Get detailed DataFrame
df = analyzer.to_dataframe()
print("\nFunctions with their docstrings:")
print(df[[function_name, docstring, args, return_annotation]])

# Find functions missing docstrings
missing_docs = analyzer.get_functions_missing_docstrings()
print("\nFunctions missing docstrings:")
for func in missing_docs:
    print(f"- {func.function_name} in {func.filepath}")


In [ ]:
df = analyzer.to_dataframe()
df


In [55]:
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Set
from pathlib import Path
import pandas as pd
import ast
from dataclasses import dataclass
import networkx as nx
import importlib
import importlib.metadata
import importlib.util
import os
import sys
from importlib.metadata import distributions, version, PackageNotFoundError

def get_stdlib_modules() -> Set[str]:
    """Get a set of standard library module names"""
    stdlib_paths = {
        os.path.dirname(os.__file__),  # Standard library path
        os.path.dirname(importlib.__file__),  # Additional standard library path
    }
    
    stdlib_modules = set()
    for path in stdlib_paths:
        if os.path.exists(path):
            for item in os.listdir(path):
                name, ext = os.path.splitext(item)
                if ext in {'.py', '.pyc'}:
                    stdlib_modules.add(name)
    
    return stdlib_modules

class DependencyInfo(BaseModel):
    name: str
    version: Optional[str] = None
    is_standard_lib: bool = False
    is_local: bool = False

    @classmethod
    def from_import(cls, module_name: str, code_root: Path) -> "DependencyInfo":
        """Create DependencyInfo from a module name"""
        root_module = module_name.split('.')[0]
        
        # Check if its a standard library module
        if root_module in get_stdlib_modules():
            return cls(
                name=root_module,
                is_standard_lib=True
            )
        
        # Check if its a local module
        local_module_path = code_root / f"{root_module}.py"
        if local_module_path.exists():
            return cls(
                name=root_module,
                is_local=True
            )
        
        # Must be an external dependency
        try:
            dep_version = version(root_module)
            return cls(
                name=root_module,
                version=dep_version
            )
        except PackageNotFoundError:
            return cls(name=root_module)


class ArgumentInfo(BaseModel):
    name: str
    type_annotation: Optional[str] = None
    default_value: Optional[str] = None

class MethodInfo(BaseModel):
    name: str
    code: str
    docstring: Optional[str] = None
    args: List[ArgumentInfo] = Field(default_factory=list)
    return_annotation: Optional[str] = None
    decorators: List[str] = Field(default_factory=list)
    is_property: bool = False
    is_classmethod: bool = False
    is_staticmethod: bool = False

class ClassInfo(BaseModel):
    name: str
    code: str
    docstring: Optional[str] = None
    methods: List[MethodInfo] = Field(default_factory=list)
    base_classes: List[str] = Field(default_factory=list)
    filepath: str
    class_variables: Dict[str, str] = Field(default_factory=dict)

class ImportInfo(BaseModel):
    module_name: str
    imported_names: List[str] = Field(default_factory=list)
    is_from_import: bool = False
    alias: Optional[str] = None


class FunctionInfo(BaseModel):
    code: str
    function_name: str
    filepath: str
    docstring: Optional[str] = None
    args: List[ArgumentInfo] = Field(default_factory=list)
    return_annotation: Optional[str] = None
    decorators: List[str] = Field(default_factory=list)

class FileInfo(BaseModel):
    path: Path
    functions: List[FunctionInfo] = Field(default_factory=list)
    classes: List[ClassInfo] = Field(default_factory=list)
    imports: List[ImportInfo] = Field(default_factory=list)
    dependencies: List[DependencyInfo] = Field(default_factory=list)

class CodeAnalyzer(BaseModel):
    code_root: Path
    def_prefixes: List[str] = Field(default=['def ' , 'async def '])
    newline: str = Field(default='\n')
    files: List[FileInfo] = Field(default_factory=list)
    dependency_graph: Optional[nx.DiGraph] = None
    
    class Config:
        arbitrary_types_allowed = True

    def _parse_argument(self, arg: ast.arg, default=None) -> ArgumentInfo:
        """Parse function/method argument information"""
        return ArgumentInfo(
            name=arg.arg,
            type_annotation=ast.unparse(arg.annotation) if arg.annotation else None,
            default_value=ast.unparse(default) if default else None
        )

    def _extract_decorators(self, node: ast.FunctionDef) -> List[str]:
        """Extract decorator information from a function/method node"""
        return [ast.unparse(decorator) for decorator in node.decorator_list]

    def _is_special_method(self, decorator_list: List[str]) -> tuple:
        """Determine if method has special decorators"""
        is_property = any(property in d for d in decorator_list)
        is_classmethod = any(classmethod in d for d in decorator_list)
        is_staticmethod = any(staticmethod in d for d in decorator_list)
        return is_property, is_classmethod, is_staticmethod

    def _extract_class_info(self, node: ast.ClassDef, code: str, filepath: str) -> ClassInfo:
        """Extract information about a class"""
        methods = []
        class_variables = {}
        
        # Get base classes
        base_classes = [ast.unparse(base) for base in node.bases]
        
        for body_item in node.body:
            if isinstance(body_item, ast.FunctionDef):
                method_code = ast.unparse(body_item)
                decorators = self._extract_decorators(body_item)
                is_property, is_classmethod, is_staticmethod = self._is_special_method(decorators)
                
                # Parse arguments
                args = []
                for i, arg in enumerate(body_item.args.args[1:] if not is_staticmethod else body_item.args.args):
                    default = body_item.args.defaults[i] if i < len(body_item.args.defaults) else None
                    args.append(self._parse_argument(arg, default))

                methods.append(MethodInfo(
                    name=body_item.name,
                    code=method_code,
                    docstring=ast.get_docstring(body_item),
                    args=args,
                    return_annotation=ast.unparse(body_item.returns) if body_item.returns else None,
                    decorators=decorators,
                    is_property=is_property,
                    is_classmethod=is_classmethod,
                    is_staticmethod=is_staticmethod
                ))
            elif isinstance(body_item, ast.AnnAssign):
                # Class variable with type annotation
                class_variables[ast.unparse(body_item.target)] = ast.unparse(body_item.annotation)
            elif isinstance(body_item, ast.Assign):
                # Class variable without type annotation
                for target in body_item.targets:
                    class_variables[ast.unparse(target)] = None

        return ClassInfo(
            name=node.name,
            code=ast.unparse(node),
            docstring=ast.get_docstring(node),
            methods=methods,
            base_classes=base_classes,
            filepath=filepath,
            class_variables=class_variables
        )

    def _extract_imports(self, node: ast.Module) -> List[ImportInfo]:
        """Extract import information from a module"""
        imports = []
        for node_item in node.body:
            if isinstance(node_item, ast.Import):
                for name in node_item.names:
                    imports.append(ImportInfo(
                        module_name=name.name,
                        alias=name.asname,
                        imported_names=[]
                    ))
            elif isinstance(node_item, ast.ImportFrom):
                if node_item.module:
                    imports.append(ImportInfo(
                        module_name=node_item.module,
                        imported_names=[name.name for name in node_item.names],
                        is_from_import=True,
                        alias=None
                    ))
        return imports

    def _analyze_dependencies(self, imports: List[ImportInfo]) -> List[DependencyInfo]:
        """Analyze dependencies from imports"""
        seen_modules = set()
        dependencies = []
        
        for imp in imports:
            module_name = imp.module_name
            if module_name in seen_modules:
                continue
                
            seen_modules.add(module_name)
            dependencies.append(
                DependencyInfo.from_import(module_name, self.code_root)
            )
        
        return dependencies
    def analyze_file(self, filepath: str) -> FileInfo:
        """Analyze a single Python file"""
        with open(filepath, 'r', encoding='utf-8') as file:
            content = file.read()
            
        try:
            tree = ast.parse(content)
            
            functions = []
            classes = []
            
            # Extract imports first
            imports = self._extract_imports(tree)
            dependencies = self._analyze_dependencies(imports)
            
            # Handle top-level nodes
            for node in tree.body:
                if isinstance(node, ast.FunctionDef):
                    functions.append(self._extract_function_info(node, filepath))
                elif isinstance(node, ast.ClassDef):
                    classes.append(self._extract_class_info(node, content, filepath))
                    
            return FileInfo(
                path=Path(filepath),
                functions=functions,
                classes=classes,
                imports=imports,
                dependencies=dependencies
            )
        except SyntaxError:
            print(f"Syntax error in file: {filepath}")
            return FileInfo(path=Path(filepath))
    def build_dependency_graph(self):
        """Build a dependency graph of the project"""
        self.dependency_graph = nx.DiGraph()
        
        for file_info in self.files:
            file_node = str(file_info.path)
            self.dependency_graph.add_node(file_node)
            
            for dep in file_info.dependencies:
                if dep.is_local:
                    # Add edge for local dependencies
                    dep_file = str(self.code_root / f"{dep.name}.py")
                    self.dependency_graph.add_edge(file_node, dep_file)

    def get_class_hierarchy(self) -> nx.DiGraph:
        """Build and return the class hierarchy graph"""
        hierarchy = nx.DiGraph()
        
        for file_info in self.files:
            for class_info in file_info.classes:
                hierarchy.add_node(class_info.name)
                for base in class_info.base_classes:
                    hierarchy.add_edge(base, class_info.name)
                    
        return hierarchy

    def get_dependency_cycles(self) -> List[List[str]]:
        """Find circular dependencies in the project"""
        if not self.dependency_graph:
            self.build_dependency_graph()
        return list(nx.simple_cycles(self.dependency_graph))

    def to_dataframe(self) -> Dict[str, pd.DataFrame]:
        """Convert the analyzed data to multiple DataFrames"""
        try:
            # Functions DataFrame
            functions_data = [
                {
                    'filepath': func.filepath,
                    'name': func.function_name,
                    'docstring': func.docstring,
                    'args': [arg.dict() for arg in func.args],
                    'return_annotation': func.return_annotation,
                    'decorators': func.decorators
                }
                for file in self.files
                for func in file.functions
            ]
            
            # Classes DataFrame
            classes_data = [
                {
                    'filepath': cls.filepath,
                    'name': cls.name,
                    'docstring': cls.docstring,
                    'methods_count': len(cls.methods),
                    'base_classes': cls.base_classes,
                    'variables_count': len(cls.class_variables),
                    'methods': [method.name for method in cls.methods],
                    'has_docstring': cls.docstring is not None
                }
                for file in self.files
                for cls in file.classes
            ]
            
            # Dependencies DataFrame
            dependencies_data = [
                {
                    'filepath': str(file.path),
                    'dependency': dep.name,
                    'version': dep.version,
                    'is_standard_lib': dep.is_standard_lib,
                    'is_local': dep.is_local
                }
                for file in self.files
                for dep in file.dependencies
            ]

            return {
                'functions': pd.DataFrame(functions_data) if functions_data else pd.DataFrame(),
                'classes': pd.DataFrame(classes_data) if classes_data else pd.DataFrame(),
                'dependencies': pd.DataFrame(dependencies_data) if dependencies_data else pd.DataFrame()
            }
        except Exception as e:
            print(f"Error creating DataFrames: {str(e)}")
            return {
                'functions': pd.DataFrame(),
                'classes': pd.DataFrame(),
                'dependencies': pd.DataFrame()
            }
    def _extract_function_info(self, node: ast.FunctionDef, filepath: str) -> FunctionInfo:
        """Extract information about a function"""
        args = []
        for i, arg in enumerate(node.args.args):
            default = node.args.defaults[i] if i < len(node.args.defaults) else None
            args.append(self._parse_argument(arg, default))

        return FunctionInfo(
            code=ast.unparse(node),
            function_name=node.name,
            filepath=filepath,
            docstring=ast.get_docstring(node),
            args=args,
            return_annotation=ast.unparse(node.returns) if node.returns else None,
            decorators=self._extract_decorators(node)
        )

    # def to_dataframe(self) -> pd.DataFrame:
    #     """
    #     Convert the analyzed data to a pandas DataFrame
    #     """
    #     rows = []
    #     for file in self.files:
    #         for func in file.functions:
    #             rows.append({
    #                 filepath: func.filepath,
    #                 function_name: func.function_name,
    #                 code: func.code,
    #                 docstring: func.docstring,
    #                 args: func.args,
    #                 return_annotation: func.return_annotation
    #             })
    #     return pd.DataFrame(rows)
    def analyze_repo(self) -> List[FileInfo]:
        """
        Analyze all Python files in the repository.
        Returns a list of FileInfo objects containing analysis results.
        """
        code_files = list(self.code_root.glob("**/*.py"))

        num_files = len(code_files)
        print(f"Total number of .py files: {num_files}")

        if num_files == 0:
            print("Verify the repository exists and code_root is set correctly.")
            return []

        self.files = []
        for code_file in code_files:
            try:
                file_info = self.analyze_file(str(code_file))
                self.files.append(file_info)
                print(f"Analyzed: {code_file}")
            except Exception as e:
                print(f"Error analyzing {code_file}: {str(e)}")

        print(f"\nAnalysis complete:")
        print(f"Files processed: {len(self.files)}")
        print(f"Classes found: {sum(len(f.classes) for f in self.files)}")
        print(f"Functions found: {sum(len(f.functions) for f in self.files)}")

        return self.files
    def _extract_function_info(self, node: ast.FunctionDef, filepath: str) -> FunctionInfo:
        """Extract information about a function"""
        args = []
        for i, arg in enumerate(node.args.args):
            default = node.args.defaults[i] if i < len(node.args.defaults) else None
            args.append(self._parse_argument(arg, default))

        return FunctionInfo(
            code=ast.unparse(node),
            function_name=node.name,
            filepath=filepath,
            docstring=ast.get_docstring(node),
            args=args,
            return_annotation=ast.unparse(node.returns) if node.returns else None,
            decorators=self._extract_decorators(node)
        )


In [56]:
# Initialize and analyze
code_root = Path(code_root)
analyzer = CodeAnalyzer(code_root=code_root)
analyzer.analyze_repo()

# Get DataFrames
dfs = analyzer.to_dataframe()

# Show class information
print("\nClasses:")
if not dfs['classes'].empty:
    print(dfs['classes'])
else:
    print("No classes found")

# Show dependencies
print("\nDependencies:")
if not dfs['dependencies'].empty:
    print(dfs['dependencies'])
else:
    print("No dependencies found")

# Find classes with missing documentation
if not dfs['classes'].empty and 'docstring' in dfs['classes'].columns:
    classes_missing_docs = dfs['classes'][dfs['classes']['docstring'].isna()]
    print("\nClasses missing documentation:")
    print(classes_missing_docs[['name', 'filepath']])
else:
    print("\nNo classes to check for documentation")

# Get statistics
print("\nProject Statistics:")
print(f"Total Files: {len(analyzer.files)}")
print(f"Total Classes: {len(dfs['classes']) if not dfs['classes'].empty else 0}")
print(f"Total Functions: {len(dfs['functions']) if not dfs['functions'].empty else 0}")


Total number of .py files: 14
Error analyzing c:\mygit\compuse\computer_use_demo\tools\base.py: 'in <string>' requires string as left operand, not type
Error analyzing c:\mygit\compuse\computer_use_demo\tools\bash.py: 1 validation error for ClassInfo
class_variables.description
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
Analyzed: c:\mygit\compuse\computer_use_demo\tools\bash_original.py
Analyzed: c:\mygit\compuse\computer_use_demo\tools\code_context_manager.py
Analyzed: c:\mygit\compuse\computer_use_demo\tools\collection.py
Error analyzing c:\mygit\compuse\computer_use_demo\tools\computer.py: 2 validation errors for ClassInfo
class_variables.COMPUTER
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
class_variables.API
  Input should be a va

In [58]:
df = analyzer.to_dataframe()
df['classes']


,filepath,name,docstring,methods_count,base_classes,variables_count,methods,has_docstring
0,c:\mygit\compuse\computer_use_demo\tools\bash_...,_BashSession,A session of a bash shell.,2,[],6,"[__init__, stop]",True
1,c:\mygit\compuse\computer_use_demo\tools\bash_...,BashTool,A tool that allows the agent to run bash comma...,2,[BaseAnthropicTool],3,"[__init__, to_params]",True
2,c:\mygit\compuse\computer_use_demo\tools\code_...,CodeFile,None,1,[],6,[to_message],False
3,c:\mygit\compuse\computer_use_demo\tools\code_...,CodeContextManager,None,10,[],0,"[__init__, update_file, get_context_messages, ...",False
4,c:\mygit\compuse\computer_use_demo\tools\colle...,ToolCollection,A collection of anthropic-defined tools.,2,[],0,"[__init__, to_params]",True
5,c:\mygit\compuse\computer_use_demo\tools\exper...,GetExpertOpinionTool,A tool takes a detailed description of the pro...,1,[BaseAnthropicTool],3,[to_params],True
6,c:\mygit\compuse\computer_use_demo\tools\playw...,WebNavigatorTool,A versatile tool that uses Playwright to inter...,5,[],3,"[__init__, to_params, _create_structured_conte...",True
7,c:\mygit\compuse\computer_use_demo\tools\windo...,WindowsNavigationTool,A tool specializing in navigating Windows and ...,4,[],3,"[__init__, _load_shortcuts, to_params, get_ses...",True


In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,
)

completion = client.chat.completions.create(

  model="nousresearch/hermes-3-llama-3.1-405b:free",
  messages=[{"content":  """The following app description is partially complete in the 
                           directory c:/repo/dish_tracker 
              
                           
              
                           There have been many additional features added to the html, but 
                           they are not all fully implemented.
              
                           
              
                           Your job is to complete the app.
              
                           
              
                           First, take a look at a journal of the steps taken to create the 
                           app, in the file 
                           C:\\mygit\\compuse\\computer_use_demo\\journal\\journal.log
              
                           
              
                           The biggest issue us that I can no longer save any new entries.
              
                           
              
                           Remove the Login functionality, If we decide to implement it it 
                           will be much later in the future.  
              
                           
              
                           If you have any questions, instead of asking the user,please get 
                           an expert opinion.
              
                               When you are done, please test the app to make sure it 
                           works. If it or any feature of it is not working, please fix 
                           it.
              
                           
              
                           If you finish and it is all working, please ask for an expert 
                           opinion on additional features you could add.
              
                           
              
                           
              
                           ## Create a web app that stores, updates and tracks the 
                           following information in a visual appealing and easy to edit 
                           way
              
                           
              
                           It is used to track the status of Dish Machine Sales and 
                           Installations.
              
                           The following info should be stored and displayed:
              
                               - Priority of Install (1-5)
              
                               - Salesperson Name
              
                               - Name of Customer
              
                               - Machine Model (Drop Down with the ability to add new 
                           models((A4, A5, D2, UC34, A6)))
              
                               - Have Verbal Commit (Date or None)
              
                               - Have Paperwork? (Date or None)
              
                               - Have Money?? (Date or None)
              
                               - Installed (Date or None)
              
                               - Status (Drop Down with options: Pending Salesperson, 
                           Pending Machine, Pending Customer, Complete)
              
                               - Notes
              
                           
              
                           The app should have the following features:
              
                               - Add new entries
              
                               - Edit existing entries
              
                               - Delete entries
              
                               - Sort by Priority, Salesperson, Customer, Model, Status
              
                               - Filter by Status, Salesperson
              
                               - Search by Customer, Salesperson, Model, Status
              
                               - Export to Excel
              
                           Other features for ease of use or visual appeal are welcome.""",
                "role": "user"}]
)
print(completion.choices[0].message.content)


In [ ]:
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
print(OPENROUTER_API_KEY)
